## Diarization for Swiss german meetings

Loosely based on [this tutorial](https://huggingface.co/learn/audio-course/en/chapter7/transcribe-meeting) using [this version of whisper small](https://huggingface.co/ss0ffii/whisper-small-german-swiss)

In [ ]:
%%capture
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install git+https://github.com/huggingface/transformers.git
!pip install --upgrade pyannote.audio
!pip install speechbrain

In [ ]:
# @title Functions definition, run once
import datetime
import dataclasses
from typing import Any

@dataclasses.dataclass
class SpeakerSegment:
  start: float
  end: float
  text: str
  speaker: str | None = None


def canonicalize_whisper(whisper_out: Any) -> list[SpeakerSegment]:
  """Extracts transcription per interval of time in absolute cents of seconds."""
  result = []
  time_stamp_reset = 0
  last_time_stamp = 0
  for chunk in whisper_out["chunks"]:
    time_stamp = chunk["timestamp"]
    text = chunk["text"]
    if time_stamp[0] < last_time_stamp:
      if time_stamp[0] == 0:
        time_stamp_reset = last_time_stamp
      time_stamp = (time_stamp[0]+time_stamp_reset, time_stamp[1]+time_stamp_reset)
    result.append(SpeakerSegment(
        start=time_stamp[0],
        end=time_stamp[1],
        text=text
    ))
    last_time_stamp = time_stamp[1]
  return result

def match_diarization_to_whisper_text(whisper_output: Any, diarization_output: Any) -> list[SpeakerSegment]:
  """Match diarization segmentes to the corresponding whisper text."""
  # Bring whisper time-stamp to absolute cents of seconds
  canonical_whisper = canonicalize_whisper(whisper_output)

  # iterate over speaker segments
  result = []
  next_segment = canonical_whisper.pop(0)
  for segment, _ in diarization_output.itertracks():
    segment_start = segment.start
    segment_end = segment.end
    segment_label = diarization_output.get_labels(segment)
    text = ""
    while next_segment.start < segment_end:
      text += next_segment.text
      if canonical_whisper:
        next_segment = canonical_whisper.pop(0)
      else:
        break
    # Drop segment if no text got assigned
    if not text:
      continue
    result.append(SpeakerSegment(
        start=segment_start,
        end=segment_end,
        text=text,
        speaker=segment_label.pop()
    ))
  return result

def pretty_print_time(time_in_sec: int) -> str:
  """return hh:mm:ss."""
  return f'{int(time_in_sec)//3600:02d}h:{int(time_in_sec)//60:02d}m:{int(time_in_sec%60):02d}s'

def pretty_print_diary(diarized_audio: list[SpeakerSegment]):
  """Print the diarization output."""
  for chunk in diarized_audio:
    chunk_start = chunk.start
    print(f'{pretty_print_time(chunk_start)} <{chunk.speaker}>: {chunk.text}')


In [ ]:
# @title You will need a huggingface token to run the models
from huggingface_hub import login
login()

Set up the model we will use for transcribing and detecting speakers.

Models available are:
* **swiss-german**: Fast model fine tuned for swiss german, not always super precise.
* **base**: generic model with a good trade-off between speed and accuracy, not fine tuned for swiss german
* **large**: slower but should be the more accurate.

In [ ]:
%%capture
# Set up fine tuned whisper to be used for the actual transcription, running on GPU
from transformers import pipeline
from pyannote.audio import Pipeline
import torch

whisper_type = {
    'swiss_german': 'ss0ffii/whisper-small-german-swiss',
    'base': 'openai/whisper-base',
    'large': 'openai/whisper-large-v3',
}
whisper_model = "large" # @param ["swiss_german", "base", "large"]
whisper_path = whisper_type[whisper_model]

if torch.cuda.is_available():
  whisper_pipeline = pipeline("automatic-speech-recognition", model=whisper_path, device=0)
  diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1")
  diarization_pipeline.to(torch.device("cuda"))
else:
  whisper_pipeline = pipeline("automatic-speech-recognition", model=whisper_path)
  diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1")

In [ ]:
# @title Upload the files that you would like to process

from google.colab import files

track = files.upload()

In [ ]:
# @title You can play the audio files using this cell
from IPython.display import Audio, display

for k in track:
  sound_file = k
  print(f'Play {sound_file}')
  display(Audio(sound_file, autoplay=True))

In [ ]:
# @title Run the diarization pipeline

num_speakers = 1 # @param {type:"integer"}
output = {}
for k in track:
  whisper_output = whisper_pipeline(f'/content/{k}', return_timestamps=True)
  diarization_output = diarization_pipeline(f'/content/{k}')
  diary = match_diarization_to_whisper_text(whisper_output, diarization_output)
  output[k] = {
      "whisper_output" : whisper_output,
      "diarization_output" : diarization_output,
      "combined" : diary
  }

for k in output:
  # print('-----------------------------')
  print(f'    File: {k}               ')
  pretty_print_diary(output[k]["combined"])
  # print('-----------------------------')